## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-18-01-15-39 +0000,nypost,Ethan Wyttenbach — and his family — ensuring L...,https://nypost.com/2026/02/17/sports/ethan-wyt...
1,2026-02-18-01-11-33 +0000,nypost,LA couple opens up coffee shop at their home a...,https://nypost.com/2026/02/17/us-news/la-coupl...
2,2026-02-18-01-11-17 +0000,nyt,Labubu Diplomacy,https://www.nytimes.com/2026/02/17/world/labub...
3,2026-02-18-01-07-42 +0000,nypost,Gavin Newsom’s begging email to supporters ahe...,https://nypost.com/2026/02/17/us-news/gavin-ne...
4,2026-02-18-01-05-49 +0000,nyt,Questions Swirl Around Russian Figure Skater i...,https://www.nytimes.com/2026/02/17/world/europ...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2482/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
178,trump,28
115,jackson,27
114,jesse,23
71,new,16
390,talks,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
300,2026-02-17-09-48-00 +0000,wsj,"Jesse Jackson, the civil-rights activist and p...",https://www.wsj.com/politics/jesse-jackson-civ...,115
108,2026-02-17-21-36-00 +0000,wsj,An immigration court dropped the Trump adminis...,https://www.wsj.com/politics/policy/judge-drop...,95
171,2026-02-17-18-23-29 +0000,nyt,"Al Sharpton, Bernice King and Trump Among Trib...",https://www.nytimes.com/2026/02/17/us/politics...,87
163,2026-02-17-19-00-49 +0000,nyt,"In Africa, Jesse Jackson Was Esteemed for His ...",https://www.nytimes.com/2026/02/17/world/afric...,79
274,2026-02-17-11-14-34 +0000,latimes,"Jesse Jackson, one of the nation's most powerf...",https://www.latimes.com/obituaries/story/2026-...,78


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
300,115,2026-02-17-09-48-00 +0000,wsj,"Jesse Jackson, the civil-rights activist and p...",https://www.wsj.com/politics/jesse-jackson-civ...
108,82,2026-02-17-21-36-00 +0000,wsj,An immigration court dropped the Trump adminis...,https://www.wsj.com/politics/policy/judge-drop...
253,74,2026-02-17-12-51-03 +0000,nypost,Warner Bros. Discovery will restart talks with...,https://nypost.com/2026/02/17/business/warner-...
152,48,2026-02-17-19-34-19 +0000,latimes,Stephen Colbert calls out CBS for blocking int...,https://www.latimes.com/entertainment-arts/bus...
220,45,2026-02-17-16-02-41 +0000,nypost,"Iran, US agree on ‘guiding principles’ of nucl...",https://nypost.com/2026/02/17/world-news/iran-...
78,42,2026-02-17-22-32-09 +0000,nyt,Lunar New Year in Photos and Video: Revelers W...,https://www.nytimes.com/2026/02/17/world/asia/...
101,40,2026-02-17-21-45-00 +0000,wsj,The Justice Department released what it says i...,https://www.wsj.com/us-news/law/what-we-know-s...
41,36,2026-02-17-23-41-00 +0000,wsj,Mamdani Warns of Nearly 10% Property-Tax Boost...,https://www.wsj.com/us-news/mamdani-warns-of-n...
19,36,2026-02-18-00-34-04 +0000,cbc,Victims in Rhode Island hockey arena shooting ...,https://www.cbc.ca/news/world/rhode-island-sho...
50,32,2026-02-17-23-19-44 +0000,nypost,Calls grow for DSA City Attorney candidate Mar...,https://nypost.com/2026/02/17/us-news/calls-gr...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
